In [ ]:
import gdown
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Download do stopwords
nltk.download('stopwords')

# Baixar o arquivo JSON
file_id = "1ej2uGwFvjeIoUgL6c_aem2348yYzLtIZ"
url = f"https://drive.google.com/uc?id={file_id}"
output = "corpus_SBIE.json"
gdown.download(url, output, quiet=False)

# Carregar o arquivo JSON
with open(output, "r", encoding="utf-8") as f:
    corpus = json.load(f)

# Analisar a estrutura do primeiro item
print("\nAnalisando estrutura do JSON...")
if isinstance(corpus, list) and len(corpus) > 0:
    first_item = corpus[0]

    if isinstance(first_item, dict):
        print("Chaves disponíveis no primeiro documento:", first_item.keys())

        # Lista de possíveis campos de texto
        text_candidates = ['texto', 'text', 'content', 'conteudo', 'body', 'abstract', 'resumo', 'article']

        # Encontrar um campo adequado
        selected_field = None
        for field in text_candidates:
            if field in first_item and isinstance(first_item[field], str):
                selected_field = field
                break

        if selected_field:
            print(f"\nCampo selecionado para extração de textos: '{selected_field}'")
            texts = [doc[selected_field] for doc in corpus if selected_field in doc]
        else:
            print("\nNenhum campo padrão encontrado. Procurando campo de texto...")
            for key, value in first_item.items():
                if isinstance(value, str):
                    selected_field = key
                    print(f"Campo selecionado: '{selected_field}'")
                    texts = [doc[selected_field] for doc in corpus if selected_field in doc]
                    break
    else:
        print("\nO JSON é uma lista direta de textos")
        texts = corpus
else:
    raise ValueError("Estrutura do JSON não reconhecida")

# Verificação final
if 'texts' not in locals() or len(texts) == 0:
    raise ValueError("Não foi possível identificar o campo de texto automaticamente. Por favor, especifique manualmente.")

print(f"\nNúmero de documentos carregados: {len(texts)}")
print(f"Exemplo do primeiro documento (trecho):\n{texts[0][:200]}...")

# Pré-processamento
stop_words = set(stopwords.words('portuguese'))

def preprocess(text):
    # Converter para minúsculas
    text = text.lower()
    # Remover caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    # Remover stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Aplicar pré-processamento
print("\nPré-processando textos...")
processed_texts = [preprocess(text) for text in texts]

# 1. One-Hot Encoding
print("\nCriando representação One-Hot...")
onehot_vectorizer = CountVectorizer(binary=True, max_features=5000)
onehot_matrix = onehot_vectorizer.fit_transform(processed_texts)
onehot_features = onehot_vectorizer.get_feature_names_out()

# 2. Bag of Words (BoW)
print("Criando representação Bag of Words...")
bow_vectorizer = CountVectorizer(max_features=5000)
bow_matrix = bow_vectorizer.fit_transform(processed_texts)
bow_features = bow_vectorizer.get_feature_names_out()

# 3. TF-IDF
print("Criando representação TF-IDF...")
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_texts)
tfidf_features = tfidf_vectorizer.get_feature_names_out()

# Função para encontrar tokens similares (COMPLETA E CORRETAMENTE DEFINIDA)
def find_similar_tokens(token, vectorizer, matrix, features, top_n=10):
    try:
        token_idx = np.where(features == token)[0][0]
    except IndexError:
        return []

    token_vec = matrix[:, token_idx].toarray().flatten()
    if np.all(token_vec == 0):
        return []

    similarities = cosine_similarity(matrix.T, token_vec.reshape(1, -1)).flatten()
    similar_indices = np.argsort(similarities)[::-1][1:top_n+1]
    similar_tokens = [(features[i], similarities[i]) for i in similar_indices if similarities[i] > 0]

    return similar_tokens

# Exemplo de comparação para alguns tokens
target_tokens = ["aprendizagem", "educação", "aluno", "tecnologia"]  # Adicione ou modifique os tokens de interesse

for token in target_tokens:
    print(f"\n=== Tokens similares a '{token}' ===")

    # One-Hot
    onehot_similar = find_similar_tokens(token, onehot_vectorizer, onehot_matrix, onehot_features)
    print(f"\nOne-Hot ({len(onehot_similar)} similares):")
    for t, sim in onehot_similar:
        print(f"{t}: {sim:.4f}")

    # BoW
    bow_similar = find_similar_tokens(token, bow_vectorizer, bow_matrix, bow_features)
    print(f"\nBoW ({len(bow_similar)} similares):")
    for t, sim in bow_similar:
        print(f"{t}: {sim:.4f}")

    # TF-IDF
    tfidf_similar = find_similar_tokens(token, tfidf_vectorizer, tfidf_matrix, tfidf_features)
    print(f"\nTF-IDF ({len(tfidf_similar)} similares):")
    for t, sim in tfidf_similar:
        print(f"{t}: {sim:.4f}")

# Análise adicional: mostrar as features mais frequentes em cada representação
def top_features(vectorizer, matrix, features, n=10):
    sums = matrix.sum(axis=0)
    indices = np.argsort(sums)[::-1][:n]
    return [(features[idx], sums[0, idx]) for idx in indices.A1]

print("\nTop features em cada representação:")
print("\nOne-Hot (mais frequentes):")
for feat, count in top_features(onehot_vectorizer, onehot_matrix, onehot_features):
    print(f"{feat}: {count}")

print("\nBoW (mais frequentes):")
for feat, count in top_features(bow_vectorizer, bow_matrix, bow_features):
    print(f"{feat}: {count}")

print("\nTF-IDF (maiores pesos médios):")
tfidf_weights = np.mean(tfidf_matrix, axis=0)
top_tfidf_indices = np.argsort(tfidf_weights)[::-1].A1[:10]
for idx in top_tfidf_indices:
    print(f"{tfidf_features[idx]}: {tfidf_weights[0, idx]:.4f}")



# Função para exibir matriz de forma legível
def display_matrix(features, matrix, title, n_docs=3, n_features=10):
    # Converter para array denso e criar DataFrame
    dense_matrix = matrix[:n_docs, :n_features].toarray()
    df = pd.DataFrame(dense_matrix,
                     columns=features[:n_features],
                     index=[f"Doc {i+1}" for i in range(n_docs)])

    print(f"\n=== {title} (Primeiros {n_docs} documentos e {n_features} features) ===")
    display(df)










[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Downloading...
From: https://drive.google.com/uc?id=1ej2uGwFvjeIoUgL6c_aem2348yYzLtIZ
To: /content/corpus_SBIE.json
100%|██████████| 13.2M/13.2M [00:00<00:00, 85.7MB/s]



Analisando estrutura do JSON...
Chaves disponíveis no primeiro documento: dict_keys(['titulo', 'informacoes_url', 'idioma', 'storage_key', 'autores', 'data_publicacao', 'resumo', 'keywords', 'referencias', 'artigo_completo_PT', 'artigo_tokenizado', 'pos_tagger', 'lema', 'dependencia'])

Campo selecionado para extração de textos: 'resumo'

Número de documentos carregados: 30
Exemplo do primeiro documento (trecho):
Feiras de Ciências são abordagens pedagógicas que colocam os es-
tudantes como protagonistas do seu processo de aprendizagem, por meio da
experimentação e pesquisa científica. No entanto, desafios sur...

Pré-processando textos...

Criando representação One-Hot...
Criando representação Bag of Words...
Criando representação TF-IDF...

=== Tokens similares a 'aprendizagem' ===

One-Hot (10 similares):
ensino: 0.6838
pode: 0.6547
processo: 0.6172
desafios: 0.5669
foco: 0.5669
avaliar: 0.5669
apre: 0.5345
testes: 0.5345
software: 0.5345
objetivo: 0.5345

BoW (10 similares):
pode: